In [260]:
import random


In [261]:
import deap
import numpy as np


In [262]:
from deap import base
from deap import creator
from deap import tools


In [263]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
'''
適合度を最適化（最大化）
個体
'''


'\n適合度を最適化（最大化）\n個体\n'

In [343]:
toolbox = base.Toolbox()
# Attribute generator(個体の作成)
toolbox.register("attr",random.randint,48,96)
toolbox.register("attr_rhythm",random.choice,0.25,0.5,1,2,4)
toolbox.register("attr_code",random.randint,70,76)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr, 64)#initRepeat　データを入れるコンテナの型、データを生成する関数、コンテナのサイズを引数ロする集団を発生させる関数
toolbox.register("rindividual", tools.initRepeat, creator.Individual, toolbox.attr_rhythm,10)
toolbox.register("cindividual", tools.initRepeat, creator.Individual, toolbox.attr_code,8)
toolbox.register("population1", tools.initRepeat, list, toolbox.individual)#メロディ
toolbox.register("population2", tools.initRepeat, list, toolbox.cindividual) #コード

In [265]:
#案１(botu)　音符の落差最小化

def evalOneMax(individual):
    score = 0
    for i in range(0,len(individual)-1):
        if (i >= len(individual)):
            break
        score -= abs(individual[i] - individual[i+1])
    score += sum(individual)
    return score,
'''
この部分を音楽理論に基づいた評価にしたい
方針１：
　制約
 　　１オクターブ以上の音の跳躍を許さない
   　キーにない音を含んだ場合減点
'''


'\nこの部分を音楽理論に基づいた評価にしたい\n方針１：\n\u3000制約\n \u3000\u3000１オクターブ以上の音の跳躍を許さない\n   \u3000キーにない音を含んだ場合減点\n'

In [266]:
#案2　調にない音を減点
def evalmel(individual):
    score = 0 
    tonic= 60
    for i in range(0,len(individual)-1):
        if (i >= len(individual)):
            break
        score -= abs(tonic-individual[i])** 2
        if(abs(individual[i]-individual[i+1])>12):
            score -=300
        if(individual[i] % 12 == 0 or 2 or 4 or 7 or 9):
            score += 300
        elif(individual[i] % 12 == 5 or 11):
            score += 100
        else:
            score *= 0
        
    return score,

In [377]:
def evalcode(cindividual):
    score = 0 
    sc = 0
    
    triad = {0:'T',1:'S',2:'T',3:'S',4:'D',5:'S',6:'D'}
    for i in range(0,len(cindividual)-1):
        if (i >= len(cindividual)):
            if(triad[(cindividual[i]) %7]  == 'T' ):
                score += 40
            break
        
        if(triad[(cindividual[i]) % 7] == triad[cindividual[i+1] % 7]):
            sc +=1
            if((cindividual[i]) % 7 == cindividual[i+1] % 7): #同じコードが連続していたら減点
                sc += 1
        if(triad[(cindividual[i]) % 7] == 'D' and triad[(cindividual[i+1]) % 7] == 'S'): #ドミナント→サブドミナントは減点
            score -= 100
        
    score -= (sc ** 2)
        
    return score,

In [374]:
toolbox.register("evaluate1", evalmel)#評価関数
toolbox.register("evaluate2", evalcode)
toolbox.register("mate", tools.cxTwoPoint)#交叉
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)#突然変異
toolbox.register("select", tools.selTournament,tournsize=3)#選択

In [375]:
#コード生成
code = list()
def code_generate(code):
    
    random.seed(40)
    # 初期の個体群を生成 N体作成する
    pop = toolbox.population2(n=100)
    
    CXPB, MUTPB, NGEN = 0.6, 0.05, 300 # 交差確率、突然変異確率、進化計算のループ回数
    '''
    この部分を、印象値入力にできないだろうか
    '''
    print("Start of evolution")

    # 初期の個体群の評価
    fitnesses = list(map(toolbox.evaluate2,pop))
    print(fitnesses)
    print('nyaan')
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # 進化計算開始
    for g in range(NGEN):
        print("-- Generation %i --" % g)
        if (g % 5 ==0):
            print("続行しますか？")
            print ('終了する場合は”quit”と入力してください。')
            is_continue = input('>>>  ')
            if (is_continue == 'quit'):
                break
            else:
                continue
                
        # 次世代の個体群
        offspring = toolbox.select(pop, len(pop))
        # 個体群のクローンを生成
        offspring = list(map(toolbox.clone, offspring))

        # 選択した個体群に交差と突然変異を適応する
        # 偶数番目と奇数番目の個体を取り出して交差
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # 適合度が計算されていない個体を集めて適合度を計算
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate2, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # 次世代群をoffspringにする
        pop[:] = offspring
        
        # すべての個体の適合度を配列にする
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    code[:] = best_ind[:]
    return code

In [379]:
#メロディ生成
melody = list()
def melody_generate(music):
    print ('作成したい曲の調の番号を入力してください。0:\'C\',1:\'C\#\',2:\'D\',3:\'D\#\',4:\'E\',5:\'F\',6:\'F\#\',7:\'G\',8:\'G\#\',9:\'A\',10:\'A\#\',11:\'H\'')
    tonic = input('>>>  ') 
    tonic = int(tonic) + 60
    print(tonic)
    
    random.seed(60)
    # 初期の個体群を生成 N体作成する
    pop = toolbox.population1(n=300)
    
    CXPB, MUTPB, NGEN = 0.6, 0.3, 300 # 交差確率、突然変異確率、進化計算のループ回数
    '''
    この部分を、印象値入力にできないだろうか
    '''
    print("Start of evolution")

    # 初期の個体群の評価
    fitnesses = list(map(toolbox.evaluate1,pop))
    print(fitnesses)
    print('nyaan')
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # 進化計算開始
    for g in range(NGEN):
        print("-- Generation %i --" % g)
        if (g % 30 ==0):
            print("続行しますか？")
            print ('終了する場合は”quit”と入力してください。')
            is_continue = input('>>>  ')
            if (is_continue == 'quit'):
                break
            else:
                continue
                
        # 次世代の個体群
        offspring = toolbox.select(pop, len(pop))
        # 個体群のクローンを生成
        offspring = list(map(toolbox.clone, offspring))

        # 選択した個体群に交差と突然変異を適応する
        # 偶数番目と奇数番目の個体を取り出して交差
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # 適合度が計算されていない個体を集めて適合度を計算
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate1, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # 次世代群をoffspringにする
        pop[:] = offspring
        
        # すべての個体の適合度を配列にする
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    music[:] = best_ind[:]
    return music

In [382]:
code_generate(code)

Start of evolution
[(-16,), (-36,), (-104,), (-104,), (-125,), (-9,), (-4,), (-100,), (-109,), (-16,), (-116,), (-101,), (-225,), (-204,), (-16,), (-164,), (-4,), (-104,), (-149,), (-300,), (-201,), (-64,), (-201,), (-116,), (-100,), (-36,), (-16,), (-116,), (-204,), (-9,), (-16,), (-116,), (-9,), (-116,), (-116,), (-1,), (-104,), (-4,), (-64,), (-104,), (-104,), (-209,), (-116,), (-4,), (-36,), (-36,), (-125,), (-9,), (-109,), (-125,), (-4,), (-136,), (-200,), (-16,), (-201,), (-116,), (-101,), (-201,), (-25,), (-149,), (-9,), (-9,), (-16,), (-209,), (-136,), (-116,), (-125,), (-104,), (-109,), (-181,), (-1,), (-104,), (0,), (-101,), (-16,), (-104,), (-16,), (-109,), (-25,), (-125,), (-104,), (-204,), (-204,), (-9,), (-109,), (-36,), (-209,), (-209,), (-125,), (-201,), (-116,), (-136,), (-9,), (-25,), (-9,), (-216,), (-4,), (-109,), (-9,), (-9,)]
nyaan
  Evaluated 100 individuals
-- Generation 0 --
続行しますか？
終了する場合は”quit”と入力してください。
>>>  quit
-- End of (successful) evolution --
Best indi

[74, 70, 75, 76, 70, 73, 72, 73]

In [383]:
melody_generate(melody)
























作成したい曲の調の番号を入力してください。0:'C',1:'C\#',2:'D',3:'D\#',4:'E',5:'F',6:'F\#',7:'G',8:'G\#',9:'A',10:'A\#',11:'H'
>>>  0
60
Start of evolution
[(-9457,), (-15122,), (-13440,), (-17878,), (-11264,), (-17928,), (-12852,), (-14485,), (-6690,), (-8807,), (-16242,), (-11581,), (-8756,), (-13294,), (-15164,), (-14962,), (-15460,), (-17684,), (-9472,), (-19243,), (-12701,), (-9734,), (-10500,), (-14452,), (-16058,), (-16388,), (-13418,), (-13277,), (-16803,), (-11243,), (-10749,), (-12849,), (-13908,), (-9171,), (-10523,), (-14276,), (-14517,), (-13335,), (-7937,), (-2562,), (-15276,), (-12955,), (-12013,), (-15812,), (-14512,), (-19697,), (-10665,), (-11262,), (-5834,), (-12721,), (-11774,), (-13633,), (-21232,), (-16503,), (-12768,), (-10815,), (-9181,), (-8802,), (-13860,), (-13119,), (-4303,), (-18967,), (-16845,), (-13921,), (-13474,), (-15697,), (-13432,), (-9851,), (-9932,), (-16977,), (-8377,), (-14193,), (-11394,), (-10689,), (-10872,), (-12323,), (-12873,), (-9755,), (-7256,), (-13017,), (-1

[70,
 54,
 63,
 53,
 56,
 55,
 60,
 56,
 60,
 59,
 71,
 64,
 64,
 72,
 64,
 62,
 56,
 57,
 64,
 70,
 67,
 65,
 65,
 55,
 58,
 64,
 62,
 56,
 74,
 62,
 72,
 73,
 63,
 66,
 62,
 57,
 55,
 54,
 60,
 57,
 58,
 59,
 62,
 57,
 67,
 70,
 65,
 55,
 55,
 52,
 57,
 52,
 63,
 61,
 61,
 63,
 55,
 48,
 55,
 56,
 62,
 70,
 72,
 76]

In [118]:


'''
ニューラルネットワークにより音楽から印象を得る部分
'''
import chainer
import chainer.links as L
import chainer.functions as F

class MLP(chainer.Chain):

    def __init__(self, n_mid_units=40, n_out=4):
        # パラメータを持つ層の登録
        super(MLP, self).__init__(
            l1=L.Linear(None, n_mid_units),
            l2=L.Linear(n_mid_units, n_mid_units),
            l3=L.Linear(n_mid_units, n_out),
        )

    def __call__(self, x):
        # データを受け取った際のforward計算を書く
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

gpu_id = 0

model = MLP()



In [ ]:
import mido

from mido import Message, MidiFile, MidiTrack

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(Message('program_change', program=12, time=0))
track.append(Message('note_on', note=64, velocity=64, time=32))
track.append(Message('note_off', note=64, velocity=127, time=32))

mid.save('new_song.mid')

In [18]:
mido.get_output_names()

['Microsoft GS Wavetable Synth 0']

In [19]:
with mido.open_output() as port:
    # 引数が空の場合はデフォルト選択。withにより終了時にポートを閉じる
    for msg in mid.play():
        port.send(msg)

In [26]:
individual?

Object `individual` not found.


In [20]:
#ディープラーニングによる入力に似た曲の生成
import sys
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import mido 

midi=[]


def get_mid_data(midiname):
    #1パートのみの入力
    mdata=mido.MidiFile('./{0}.mid'.format(midiname))

    for msg in mdata.tracks[1]:
        if msg.type in ['note_on']:
            midi.append(msg.note)
    return mdata

def each_cons(x, size):
    return [x[i:i+size] for i in range(len(x)-size+1)]

class RNN1(Chain):
    def __init__(self):
        super(RNN1, self).__init__(
            embed=L.EmbedID(127, 100),
            mid=L.LSTM(100, 50),  # the first LSTM layer
            out=L.Linear(50, 130),  # the feed-forward output layer
        )

    def reset_state(self):
        self.mid.reset_state()

    def __call__(self, input_vector):
        # Given the current word ID, predict the next word.
        x = self.embed(input_vector)
        h = self.mid(x)
        y = self.out(h)
        return y

rnn = RNN1()
model = L.Classifier(rnn)
optimizer = optimizers.SGD()
optimizer.setup(model)

get_mid_data('BWV772')#いろいろ入力

loss = 0
count = 0
seqlen = len(midi[1:])
for x in range(100):
    print (x)
    rnn.reset_state()
    for cur_event,next_event in each_cons(midi,2):
        cur_event, next_event = np.asarray([cur_event]).astype(np.int32), np.asarray([next_event]).astype(np.int32)
        loss += model(cur_event, next_event)
        count += 1
        if count % 30 == 0 or count == seqlen:
            #print loss.data
            model.zerograds()
            loss.backward()
            loss.unchain_backward()
            optimizer.update()


model.predictor.reset_state()
l = list()
prev_event = chainer.Variable(np.array([64], np.int32))
prob = F.softmax(model.predictor(prev_event))
l.append(59)

for i in range(300):
    prob = F.softmax(model.predictor(prev_event))
    probability = prob.data[0].astype(np.float64)
    probability /= np.sum(probability)
    index = np.random.choice(range(len(probability)), p=probability)

    l.append(index)

    prev_event = chainer.Variable(np.array([index], dtype=np.int32))

print (l)
print (len(l))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[59, 60, 62, 64, 65, 62, 64, 74, 67, 72, 71, 69, 67, 71, 74, 69, 71, 72, 74, 71, 69, 72, 71, 74, 72, 76, 74, 72, 71, 69, 69, 71, 67, 74, 72, 71, 69, 67, 74, 66, 69, 71, 72, 74, 71, 69, 71, 74, 72, 81, 80, 83, 79, 77, 76, 74, 72, 68, 69, 81, 77, 76, 74, 72, 74, 72, 71, 69, 69, 81, 41, 65, 79, 120, 43, 69, 71, 62, 64, 65, 62, 64, 65, 62, 64, 60, 67, 72, 71, 69, 71, 67, 74, 72, 69, 71, 74, 72, 69, 71, 69, 62, 64, 67, 66, 69, 71, 72, 74, 71, 72, 74, 71, 69, 66, 68, 64, 76, 74, 72, 76, 74, 73, 74, 76, 77, 69, 71, 74, 72, 71, 74, 73, 76, 74, 73, 74, 76, 77, 81, 79, 77, 76, 74, 77, 76, 74, 72, 74, 76, 77, 79, 81, 83, 84, 81, 83, 79, 84, 122, 79, 76, 77, 76, 74, 72, 74, 76, 77, 74, 76, 72, 81, 80, 83, 81, 76,

In [341]:
import pretty_midi as pm
#LSTMが出力したノート番号の配列を元にMIDI楽曲ファイルを作る


def makemidi(mlist,clist,bpm,mname):
    print ('作成したい曲の調の番号を入力してください。0:\'C\',1:\'C\#\',2:\'D\',3:\'D\#\',4:\'E\',5:\'F\',6:\'F\#\',7:\'G\',8:\'G\#\',9:\'A\',10:\'A\#\',11:\'H\'')
    tonic = input('>>>  ')
    tonic = int(tonic) + 60
    print(tonic)
    song = pm.PrettyMIDI()             #楽曲ファイルを生成
    piano = pm.Instrument(program=1)   #音色にピアノをセット
    s=0                                #時間の初期化
    delta = 60 / bpm
    
    harmony = list()
    
    tonedic = {0:0,1:2,2:4,3:5,4:7,5:9,6:11}
    
    for j in range(len(clist)):
        if(clist[j]%7 == 0 or 3 or 4):
            harmony.append([(tonedic[clist[j] % 7]+ tonic),(tonedic[clist[j] % 7]+ tonic + 4),(tonedic[clist[j] % 7]+ tonic + 7)])
        elif(clist[j]%7 == 1 or 2 or 5):
            harmony.append([(tonedic[clist[j] % 7]+ tonic),(tonedic[clist[j] % 7]+ tonic + 3),(tonedic[clist[j] % 7]+ tonic + 7)])
        else:
            harmony.append([(tonedic[clist[j] % 7]+ tonic),(tonedic[clist[j] % 7]+ tonic + 3),(tonedic[clist[j] % 7]+ tonic + 6)])
        print(harmony) 
        harmony.reverse()
        p1=harmony.pop()
        for k in range(3):
            note=pm.Note(velocity =48, pitch = p1.pop(), start =s, end = s + delta * 2) 
            piano.notes.append(note)
        s += delta*4
        print(s)
        print(delta)
        print(len(mlist))
        print(note)
        
    
    s=0
    
    for i in range(len(mlist)):
        print('u')
        if mlist[i] > 127:   #ノート番号が１２７を超えた場合休符にする
            mlist[i] = 0
        
        note=pm.Note(velocity =64, pitch = mlist[i], start =s, end = s + delta * 0.5) 
        s += delta*0.5
        print(s)
        print(delta)
        print(len(mlist))
        piano.notes.append(note)

    
    song.instruments.append(piano)

    song.write('{0}.mid'.format(mname))#song.midに書き出し
    print('Success to make {0}.mid !!\n'.format(mname))

In [208]:
music?

Object `music` not found.


In [312]:
import pandas as pd
import mido

mid = mido.MidiFile('./ga1.mid')
scale = ('C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'H')
on_notes = dict()
delta = 240
tmp_score = []
mid.print_tracks()

for track in mid.tracks:
    e_time = 0
    for msg in track:
        e_time += msg.time
        if msg.type == 'note_on':
            if msg.note in on_notes: raise
            on_notes[msg.note] = e_time
        elif msg.type == 'note_off':
            if not msg.note in on_notes: raise
            on_time = on_notes.pop(msg.note)
            dur = (e_time - on_time) / delta
            tmp_score.append([msg.note, 
                              scale[msg.note%12], 
                              msg.note//12, 
                              dur, 
                              msg.velocity, 
                              msg.channel, 
                              on_time / delta])

score = pd.DataFrame(tmp_score, columns=['noteID', 'note','octave','dur','vel','ch','time'])
score

=== Track 0
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message set_tempo tempo=500000 time=0>
<meta message end_of_track time=1>
=== Track 1
<message program_change channel=0 program=1 time=0>
<message note_on channel=0 note=60 velocity=48 time=0>
<message note_on channel=0 note=60 velocity=48 time=0>
<message note_on channel=0 note=60 velocity=48 time=0>
<message note_on channel=0 note=60 velocity=64 time=0>
<message note_on channel=0 note=62 velocity=48 time=0>
<message note_on channel=0 note=64 velocity=48 time=0>
<message note_on channel=0 note=67 velocity=48 time=0>
<message note_on channel=0 note=69 velocity=48 time=0>
<message note_on channel=0 note=69 velocity=48 time=0>
<message note_on channel=0 note=60 velocity=0 time=110>
<message note_on channel=0 note=67 velocity=64 time=0>
<message note_on channel=0 note=67 velocity=0 time=110>
<message note_on channel=0 note=68 velocity=64 time=0>
<message note_

RuntimeError: No active exception to reraise

In [384]:
makemidi(melody,code,120,'300gen')


作成したい曲の調の番号を入力してください。0:'C',1:'C\#',2:'D',3:'D\#',4:'E',5:'F',6:'F\#',7:'G',8:'G\#',9:'A',10:'A\#',11:'H'
>>>  0
60
[[67, 71, 74]]
2.0
0.5
64
Note(start=0.000000, end=1.000000, pitch=67, velocity=48)
[[60, 64, 67]]
4.0
0.5
64
Note(start=2.000000, end=3.000000, pitch=60, velocity=48)
[[69, 73, 76]]
6.0
0.5
64
Note(start=4.000000, end=5.000000, pitch=69, velocity=48)
[[71, 75, 78]]
8.0
0.5
64
Note(start=6.000000, end=7.000000, pitch=71, velocity=48)
[[60, 64, 67]]
10.0
0.5
64
Note(start=8.000000, end=9.000000, pitch=60, velocity=48)
[[65, 69, 72]]
12.0
0.5
64
Note(start=10.000000, end=11.000000, pitch=65, velocity=48)
[[64, 68, 71]]
14.0
0.5
64
Note(start=12.000000, end=13.000000, pitch=64, velocity=48)
[[65, 69, 72]]
16.0
0.5
64
Note(start=14.000000, end=15.000000, pitch=65, velocity=48)
u
0.25
0.5
64
u
0.5
0.5
64
u
0.75
0.5
64
u
1.0
0.5
64
u
1.25
0.5
64
u
1.5
0.5
64
u
1.75
0.5
64
u
2.0
0.5
64
u
2.25
0.5
64
u
2.5
0.5
64
u
2.75
0.5
64
u
3.0
0.5
64
u
3.25
0.5
64
u
3.5
0.5
64
u
3.75
0.5
64


In [167]:
makemidi(l,0.6,'invention772')

NameError: name 'l' is not defined

In [144]:
i?

In [33]:
print ('何か文字を入力してください。')
test = input('>>>  ')
if (test=='hello,world'):
    print(test)

何か文字を入力してください。
>>>  hello,world
hello,world


In [86]:
score= 100
individual = list()
individual.append(3)
individual.append(4)
individual.append(9)
score += abs(individual[1] - individual[2]) 
print(score)

105


In [44]:
list([4])

[4]

In [329]:
hi = [[3,4,5],[6,7,8],[2,7,0]]

In [332]:
m =hi.enqueue()

AttributeError: 'list' object has no attribute 'enqueue'

In [331]:
m.pop()

0